## This notebook shows how to use Orchestrator APIs for user experiments

In [ ]:
import os
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
import json

In [ ]:
slice_name='Slice-sts-mixed'

In [ ]:
fabric_rc_location=os.environ['HOME']+"/work/fabric_config/fabric_rc"
if os.path.exists(fabric_rc_location):
    with open(fabric_rc_location, 'r') as f:
        for line in f:
            if line.startswith('export'):
                os.environ[line.split('=')[0].split('export')[1].strip()] = line.split('=')[1].strip()

In [ ]:
ssh_key_file_priv=os.environ["FABRIC_SLICE_PRIVATE_KEY_FILE"]
ssh_key_file_pub=os.environ["FABRIC_SLICE_PUBLIC_KEY_FILE"]

ssh_key_pub = None
with open (ssh_key_file_pub, "r") as myfile:
    ssh_key_pub=myfile.read()
    ssh_key_pub=ssh_key_pub.strip()

## Create Slice Manager Object
Users can request tokens with different Project and Scopes by altering `project_name` and `scope` parameters in the refresh call below.

In [ ]:
slice_manager = SliceManager()

### Orchestrator API example to query for available resources

In [ ]:
status, advertised_topology = slice_manager.resources()

print(f"Status: {status}")
if status == Status.OK:
    print(f"Toplogy: {advertised_topology}")
else:
    print(f"Error: {advertised_topology}")

In [ ]:
if status == Status.OK:
    advertised_topology.draw()

## Create Slice
Users should be able to ping between nodes.

In [ ]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, ComponentModelType, ServiceType, Labels
from ipaddress import IPv4Network

# Create topology
t = ExperimentTopology()

# Add node
n1 = t.add_node(name='n1', site='UKY')

# Set capacities
cap = Capacities(core=2, ram=6, disk=10)
labels = Labels(instance_parent="renc-w2.fabric-testbed.net")
# Set Properties
n1.set_properties(capacities=cap, image_type='qcow2', image_ref='default_rocky_8')

# Add PCI devices
n1.add_component(ctype=ComponentType.NVME, model='P4510', name='c1')

# Add node
n2 = t.add_node(name='n2', site='RENC')

# Set properties
n2.set_properties(capacities=cap, image_type='qcow2', image_ref='default_rocky_8')


# Shared Cards
n1.add_component(model_type=ComponentModelType.SmartNIC_ConnectX_6, name='n1-nic1')
n2.add_component(model_type=ComponentModelType.SmartNIC_ConnectX_6, name='n2-nic1')

# L2STS Service
t.add_network_service(name='sts1', nstype=ServiceType.L2STS,
                      interfaces=t.interface_list)

# Auto Configure: Setup IP Addresses
network = IPv4Network("192.168.10.0/24")
ips = network.hosts()
next(ips)

for ifs in t.interface_list:
    # Set the flags with auto_config to ensure IP Addresses are configured by CF
    ifs.flags = Flags(auto_config=True)
    if_labels = ifs.get_property(pname="labels")
    if_labels = Labels.update(if_labels, ipv4=str(next(ips)), ipv4_subnet=str(network))
    #if_labels = Labels.update(if_labels, vlan="200", ipv4=str(next(ips)))
    ifs.set_properties(labels=if_labels)

# Generate Slice Graph
slice_graph = t.serialize()


# Request slice from Orchestrator
status, reservations = slice_manager.create(slice_name=slice_name, slice_graph=slice_graph, ssh_key=ssh_key_pub)

print("Response Status {}".format(status))
if status == Status.OK:
    print("Reservations created {}".format(reservations))
    slice_id = reservations[0].slice_id
else:
    print(f"Failure: {reservations}")

## Query Slices

In [ ]:
status, slices = slice_manager.slices()

print("Response Status {}".format(status))
if status == Status.OK:    
    slice_object=list(filter(lambda s: s.slice_id == slice_id, slices))[0]
    print(slice_object)
else:
    print(f"Failure: {slices}")

## Query Slivers

In [ ]:
status, slivers = slice_manager.slivers(slice_object=slice_object)

print("Response Status {}".format(status))
if status == Status.OK:
    for s in slivers:
        print("=====================")
        print("ReservationId {}".format(s.reservation_id))
        print("State         {}".format(s.reservation_state))
        print("Pending       {}".format(s.pending_state))
        print("Join State    {}".format(s.join_state))
        print("Sliver Type   {}".format(s.sliver_type))
        
        print("Sliver        {}".format(s.sliver))
        if s.sliver_type == "NodeSliver":
            print()
            print("Management IP {}".format(s.sliver.management_ip))
            for d in s.sliver.attached_components_info.devices.values():
                print("-------------------------")
                print("COMPONENT        {}".format(d))
                print("-------------------------")
        else:
            for d in s.sliver.interface_info.interfaces.values():
                print("-------------------------")
                print("INTERFACE        {}".format(d))
                print("-------------------------")
                
        print("Notices       {}".format(s.notices))
        print("=====================")
else:
    print(f"Failure: {slivers}")

## Delete Slice

In [ ]:
status, result = slice_manager.delete(slice_object=slice_object)

print("Response Status {}".format(status))
print("Response received {}".format(result))